In [2]:
from Parameters.TrainingParameters import TrainingParameters
from Utils.PrintUtils import print_big
from CustomExceptions.Exceptions import NotSupportedError
from json import load
from typing import Iterable, Union
import torch
import pandas as pd
from torch.jit import Error
from torch.utils.data import Dataset
import pm4py
from datetime import timedelta
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from Utils.Constants import Constants
from Utils.FileUtils import file_exists
import json
import os
from Parameters.Enums import PreprocessedDfType

/Users/jrhs/.pyenv/versions/api-dev/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [44]:
filePath = TrainingParameters.bpi_2012_path

In [45]:
log = pm4py.read_xes(filePath)

parsing log, completed traces :: 100%|██████████| 13087/13087 [00:05<00:00, 2360.15it/s]


In [46]:
flattern_log: list[dict[str, any]]  = ([{**event,
                                                'caseid': trace.attributes['concept:name']}
                                               for trace in log for event in trace])


In [71]:
df = pd.DataFrame(flattern_log)

In [59]:
df_A = df[df["concept:name"].str.startswith("A_")]

In [72]:
df_O = df[df["concept:name"].str.startswith("O_")]

In [73]:
df_OW = df[df["concept:name"].str.startswith("O_") | df["concept:name"].str.startswith("W_") ]

In [74]:
df_W = df[df["concept:name"].str.startswith("W_")]

In [75]:
df_W

,org:resource,lifecycle:transition,concept:name,time:timestamp,caseid
3,112,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,173688
4,NaN,START,W_Completeren aanvraag,2011-10-01 11:36:46.437000+02:00,173688
10,NaN,SCHEDULE,W_Nabellen offertes,2011-10-01 11:45:11.554000+02:00,173688
11,NaN,COMPLETE,W_Completeren aanvraag,2011-10-01 11:45:13.917000+02:00,173688
12,NaN,START,W_Nabellen offertes,2011-10-01 12:15:41.290000+02:00,173688
...,...,...,...,...,...
262192,11119,START,W_Nabellen offertes,2012-03-10 12:46:22.700000+01:00,214373
262193,11119,COMPLETE,W_Nabellen offertes,2012-03-10 12:50:54.881000+01:00,214373
262196,112,SCHEDULE,W_Afhandelen leads,2012-02-29 23:52:01.287000+01:00,214376
262197,11169,START,W_Afhandelen leads,2012-03-01 09:26:46.736000+01:00,214376


In [5]:
df["name_and_transition"] = df["concept:name"] + \
            "_" + df["lifecycle:transition"]
df = df[['time:timestamp', 'name_and_transition', "caseid"]]

In [6]:
df.head(5)

,time:timestamp,name_and_transition,caseid
0,2011-10-01 00:38:44.546000+02:00,A_SUBMITTED_COMPLETE,173688
1,2011-10-01 00:38:44.880000+02:00,A_PARTLYSUBMITTED_COMPLETE,173688
2,2011-10-01 00:39:37.906000+02:00,A_PREACCEPTED_COMPLETE,173688
3,2011-10-01 00:39:38.875000+02:00,W_Completeren aanvraag_SCHEDULE,173688
4,2011-10-01 11:36:46.437000+02:00,W_Completeren aanvraag_START,173688


In [33]:
newData = []
for case, group in df.groupby('caseid'):
            group.sort_values("time:timestamp", ascending=True, inplace=True)
            strating_time = group.iloc[0]["time:timestamp"] - \
                timedelta(microseconds=1)
            ending_time = group.iloc[-1]["time:timestamp"] + \
                timedelta(microseconds=1)

            # print("Before to dict")
            traces = group.to_dict('records')
            # print("After to dict")


            # Add start and end tags.
            traces.insert(
                0, {"caseid": case, "time:timestamp": strating_time, "name_and_transition": Constants.SOS_VOCAB})
            traces.append(
                {"caseid": case, "time:timestamp": ending_time, "name_and_transition": Constants.EOS_VOCAB})
            # print("before extend to dict")
            newData.extend(traces)
            # print("After extend to dict")


In [35]:
df = pd.DataFrame(newData)
df['name_and_transition'] = df['name_and_transition'].astype(
            'category')

In [36]:
df.head(5)

,caseid,time:timestamp,name_and_transition
0,173688,2011-10-01 00:38:44.545999+02:00,<SOS>
1,173688,2011-10-01 00:38:44.546000+02:00,A_SUBMITTED_COMPLETE
2,173688,2011-10-01 00:38:44.880000+02:00,A_PARTLYSUBMITTED_COMPLETE
3,173688,2011-10-01 00:39:37.906000+02:00,A_PREACCEPTED_COMPLETE
4,173688,2011-10-01 00:39:38.875000+02:00,W_Completeren aanvraag_SCHEDULE


In [41]:
vocab_dict: dict[str, int] = {}
for i, cat in enumerate(df['name_and_transition'].cat.categories):
    vocab_dict[cat] = i+1
vocab_dict[Constants.PAD_VOCAB] = 0


In [42]:
df = df[df["name_and_transition"].startswith("")]

AttributeError: 'Series' object has no attribute 'start_with'

In [81]:
from Data.BPI2012Dataset import BPI2012Dataset, ActivityType

ac_list = [ActivityType.A, ActivityType.O, ActivityType.W]

In [91]:
df[any([df["concept:name"].str.startswith(a.value) for a in ac_list])]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [108]:
st = [df["concept:name"].str.startswith(a.value) for a in ac_list]

In [112]:
df[[any([a, b, c]) for a, b, c in zip(*([df["concept:name"].str.startswith(a.value) for a in ac_list]))]]
    


,org:resource,lifecycle:transition,concept:name,time:timestamp,caseid
0,112,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,173688
1,112,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,173688
2,112,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,173688
3,112,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,173688
4,NaN,START,W_Completeren aanvraag,2011-10-01 11:36:46.437000+02:00,173688
...,...,...,...,...,...
262195,112,COMPLETE,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423000+01:00,214376
262196,112,SCHEDULE,W_Afhandelen leads,2012-02-29 23:52:01.287000+01:00,214376
262197,11169,START,W_Afhandelen leads,2012-03-01 09:26:46.736000+01:00,214376
262198,11169,COMPLETE,A_DECLINED,2012-03-01 09:27:37.118000+01:00,214376


In [103]:
c

False

In [117]:
sorted([ a.value for a in ac_list ], key= str.lower)

['A_', 'O_', 'W_']

In [121]:
"".join(sorted([ a.value for a in ac_list ], key= str.lower))

'A_O_W_'

In [3]:
from Parameters.Enums import ActivityType

In [4]:
ac = [ActivityType.O, ActivityType.A, ActivityType.W]

In [7]:
ac_str = str(ac)

In [2]:
import json

In [1]:
from Utils.SaveUtils import get_json_dict

In [11]:
get_json_dict(ac)

TypeError: vars() argument must have __dict__ attribute

In [14]:
string_a = [ a.value  for a in ac ]

In [7]:
get_json_dict(string_a)

TypeError: vars() argument must have __dict__ attribute

In [8]:
dumped =  json.dumps(string_a)

In [9]:
dumped

'["O", "A", "W"]'

In [10]:
str_list =  json.loads(dumped)

In [11]:
str_list

['O', 'A', 'W']

In [13]:
[ActivityType[i] for i in str_list]

[<ActivityType.O: 'O'>, <ActivityType.A: 'A'>, <ActivityType.W: 'W'>]

In [20]:
isinstance(ac, list)

True

In [7]:
from enum import Enum

In [8]:
def get_json_dict(t):
    if isinstance(t,  Enum):
        return t.value

    if isinstance(t, str):
        return t

    if isinstance(t, int) | isinstance(t, float):
        return t

    json_dict = {}

    # If it's dict
    for k, v in vars(t).items():
        if not k.endswith("__"):
            if type(v) == type:
                json_dict[k] = get_json_dict(v)
            elif isinstance(v, Enum):
                json_dict[k] = v.value
            elif isinstance(v, list):
                json_dict[k] = [get_json_dict(v_i) for v_i in v]
            else:
                json_dict[k] = v
    return json_dict

In [9]:
get_json_dict(TrainingParameters)

{'bpi_2012_path': './datasets/event_logs/BPI_Challenge_2012.xes',
 'preprocessed_bpi_2012_folder_path': './datasets/preprocessed/BPI_Challenge_2012',
 'preprocessed_df_type': 1,
 'load_model_folder_path': None,
 'load_optimizer': True,
 'dataset': 'BPI2012',
 'BPI2012_include_types': ['O', 'W'],
 'model': 'BaseLineLSTMModel',
 'optimizer': 'Adam',
 'loss': 'CrossEntropy',
 'stop_epoch': 100,
 'batch_size': 64,
 'train_test_split_portion': [0.8, 0.1],
 'verbose_freq': 100,
 'run_validation_freq': 200,
 'dataset_split_seed': 12345,
 'max_eos_predicted_length': 50,
 'OptimizerParameters': {'learning_rate': 0.005,
  'l2': 1e-06,
  'scheduler': 'StepScheduler',
  'lr_scheduler_step': 1000,
  'lr_scheduler_gamma': 0.85,
  'SGD_momentum': 0.9},
 'BaselineLSTMModelParameters': {'embedding_dim': 256,
  'lstm_hidden': 512,
  'dropout': 0.2,
  'num_lstm_layers': 2}}

In [3]:
from Parameters.TrainingParameters import TrainingParameters

In [4]:
def get_json_dict(t):
    if isinstance(t,  Enum):
        return t.value

    if isinstance(t, str):
        return t

    json_dict = {}

    print(t)

    # If it's dict
    for k, v in vars(t).items():
        if not k.endswith("__"):
            if type(v) == type:
                json_dict[k] = get_json_dict(v)
            elif isinstance(v, Enum):
                json_dict[k] = v.value
            elif isinstance(v, list):
                json_dict[k] = [get_json_dict(v_i) for v_i in v]
            else:
                json_dict[k] = v
    return json_dict

In [40]:
ActivityType.O is Enum

False

In [42]:
isinstance("dsadas" ,str)

True